In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns



In [ ]:
class Economy:
    def __init__(self, productivity0, labor0, capital0, a, b, CL, CK, DL, DK, rho):
        self.productivity = productivity0
        self.labor = labor0
        self.capital = capital0
        self.a = a
        self.b = b
        self.CL = CL
        self.CK = CK
        self.DL = DL
        self.DK = DK
        self.rho = np.ones_like(labor0) * rho
        
        self.steps = 0
        self.dt = 0.001
        
        self.labor_hist = []
        self.capital_hist = []
        self.production_hist = []
        
    def update(self):
        capacity = self.production * 10
        
        # self.labor = self.labor + self.dt * self.a * self.labor * (1 - self.labor / capacity) - self.dt * self.labor * np.dot(self.CL, self.capital)
        # self.capital = self.capital - self.dt * self.b * self.capital + self.dt * self.capital * np.dot(self.CK, self.labor) * (1 - self.capital / capacity)
        
        labor = np.copy(self.labor)
        labor += self.dt * self.a * self.labor * (1 - self.labor / capacity) - self.dt * self.labor * np.dot(self.CL, self.capital)
        labor -= self.dt * self.labor * np.dot(self.DL, self.labor)
        
        capital = np.copy(self.capital)
        capital += self.dt * self.capital * np.dot(self.CK, self.labor) * (1 - self.capital / capacity) - self.dt * self.b * self.capital
        capital -= self.dt * self.capital * np.dot(self.DK, self.capital) 
        
        self.labor = labor
        self.capital = capital
        self.productivity += self.dt * self.productivity * 0.02
        self.steps += 1
        self.labor_hist.append(self.labor)
        self.capital_hist.append(self.capital)
        self.production_hist.append(self.production)
    
    @property
    def production(self):
        return self.productivity * self.labor ** self.rho * self.capital ** (1 - self.rho)

In [ ]:
N = 100
T = 50000
productivity0 = np.linspace(1, 3, N)
labor0 = np.ones(N) * 1
capital0 = np.ones(N) * 1
a = np.ones(N) * 1
b = np.ones(N) * 1
CL = np.diag(np.ones(N))
CK = CL
DL = np.zeros((N, N))
DK = np.zeros((N, N))
# DL = np.diag(np.ones(N)) / 100
# DK = np.diag(np.ones(N)) / 100
rho = 0.5

econ = Economy(productivity0, labor0, capital0, a, b, CL, CK, DL, DK, rho)
total_labor_results = []
total_capital_results = []
total_production_results = []

for i in range(T):
    econ.update()
    total_labor_results.append(np.sum(econ.labor))
    total_capital_results.append(np.sum(econ.capital))
    total_production_results.append(np.sum(econ.production))

production_results = np.concatenate(econ.production_hist)
data = pd.DataFrame(production_results, columns=['production'])
sector = [i for i in range(N)] * T
data['sector'] = sector
steps = []
for t in range(T):
    steps += [t] * N
data['steps'] = steps
sns.set_style('whitegrid')
sns.lineplot(x='steps', y='production', hue='sector', data=data)

In [ ]:
labor_results = np.concatenate(econ.labor_hist)
data = pd.DataFrame(labor_results, columns=['labor'])
sector = [i for i in range(N)] * T
data['sector'] = sector
steps = []
for t in range(T):
    steps += [t] * N
data['steps'] = steps
sns.set_style('whitegrid')
sns.lineplot(x='steps', y='labor', hue='sector', data=data)

In [ ]:
capital_results = np.concatenate(econ.capital_hist)
data = pd.DataFrame(capital_results, columns=['capital'])
sector = [i for i in range(N)] * T
data['sector'] = sector
steps = []
for t in range(T):
    steps += [t] * N
data['steps'] = steps
sns.set_style('whitegrid')
sns.lineplot(x='steps', y='capital', hue='sector', data=data)

In [ ]:
data = np.concatenate([total_labor_results, total_capital_results, total_production_results])
data = pd.DataFrame(data, columns=['quantity'])
kind = ['labor'] * T + ['capital'] * T + ['production'] * T
data['kind'] = kind
steps = [t for t in range(T)] + [t for t in range(T)] + [t for t in range(T)]
data['steps'] = steps

sns.set_style('whitegrid')
sns.lineplot(x='steps', y='quantity', hue='kind', data=data)

In [ ]:
# T = 100000
# CL = np.tril(np.ones(N))
# CL /= np.sum(CL, axis=1, keepdims=True)
productivity0 = np.ones(N) * 2.
CL = np.random.uniform(size=(N, N))
CL = np.where(CL > 0.8, 1., 0.)
# CL /= np.sqrt(np.sum(CL))
CL /= np.sum(CL, axis=1, keepdims=True)
CK = CL.T
# CK = np.random.uniform(size=(N, N))
# CK = np.where(CK > 0.8, CK, 0.) / 20

# DL = np.diag(np.ones(N)) + np.random.normal(size=(N, N)) / 100
# DK = np.diag(np.ones(N)) + np.random.normal(size=(N, N)) / 100
# DL = np.random.uniform(size=(N, N))
# DL = np.where(DL > 0.8, 1., 0.) / 40
# DL = DL / np.sum(DL, axis=1, keepdims=True)
# DK = DL
# DK = np.random.uniform(size=(N, N))
# DK = np.where(DK > 0.8, 1., 0.) / 40
# DK = DK / np.sum(DK, axis=1, keepdims=True)

econ = Economy(productivity0, labor0, capital0, a, b, CL, CK, DL, DK, rho)
total_labor_results = []
total_capital_results = []
total_production_results = []

for i in range(T):
    econ.update()
    total_labor_results.append(np.sum(econ.labor))
    total_capital_results.append(np.sum(econ.capital))
    total_production_results.append(np.sum(econ.production))

production_results = np.concatenate(econ.production_hist)
data = pd.DataFrame(production_results, columns=['production'])
sector = [i for i in range(N)] * T
data['sector'] = sector
steps = []
for t in range(T):
    steps += [t] * N
data['steps'] = steps
sns.set_style('whitegrid')
sns.lineplot(x='steps', y='production', hue='sector', data=data)


In [ ]:
labor_results = np.concatenate(econ.labor_hist)
data = pd.DataFrame(labor_results, columns=['labor'])
data['sector'] = sector
data['steps'] = steps
sns.set_style('whitegrid')
sns.lineplot(x='steps', y='labor', hue='sector', data=data)


In [ ]:
capital_results = np.concatenate(econ.capital_hist)
data = pd.DataFrame(capital_results, columns=['capital'])
data['sector'] = sector
data['steps'] = steps
sns.set_style('whitegrid')
sns.lineplot(x='steps', y='capital', hue='sector', data=data)


In [ ]:
data = np.concatenate([total_labor_results, total_capital_results, total_production_results])
data = pd.DataFrame(data, columns=['quantity'])
kind = ['labor'] * T + ['capital'] * T + ['production'] * T
data['kind'] = kind
steps = [t for t in range(T)] + [t for t in range(T)] + [t for t in range(T)]
data['steps'] = steps

sns.set_style('whitegrid')
sns.lineplot(x='steps', y='quantity', hue='kind', data=data)

In [ ]:
T = 50000
productivity0 = np.ones(N) * 2.
# CL = np.random.uniform(size=(N, N))
# CL = np.where(CL > 0.8, 1., 0.)
# CL /= np.sqrt(np.sum(CL))
# CL /= np.sum(CL, axis=1, keepdims=True)
# CK = CL.T
# CK = np.random.uniform(size=(N, N))
# CK = np.where(CK > 0.8, CK, 0.) / 20

# DL = np.diag(np.ones(N)) + np.random.normal(size=(N, N)) / 100
# DK = np.diag(np.ones(N)) + np.random.normal(size=(N, N)) / 100
DL = np.random.uniform(size=(N, N))
DL = np.where(DL > 0.8, 1., 0.)
DL = DL / np.sum(DL, axis=1, keepdims=True)
# DK = DL
DK = np.random.uniform(size=(N, N))
DK = np.where(DK > 0.8, 1., 0.)
DK = DK / np.sum(DK, axis=1, keepdims=True)

econ = Economy(productivity0, labor0, capital0, a, b, CL, CK, DL, DK, rho)
total_labor_results = []
total_capital_results = []
total_production_results = []

for i in range(T):
    econ.update()
    total_labor_results.append(np.sum(econ.labor))
    total_capital_results.append(np.sum(econ.capital))
    total_production_results.append(np.sum(econ.production))

sector = [i for i in range(N)] * T
steps = []
for t in range(T):
    steps += [t] * N

production_results = np.concatenate(econ.production_hist)
data = pd.DataFrame(production_results, columns=['production'])
data['sector'] = sector
data['steps'] = steps
sns.set_style('whitegrid')
sns.lineplot(x='steps', y='production', hue='sector', data=data)


In [ ]:
labor_results = np.concatenate(econ.labor_hist)
data = pd.DataFrame(labor_results, columns=['labor'])
data['sector'] = sector
data['steps'] = steps
sns.set_style('whitegrid')
sns.lineplot(x='steps', y='labor', hue='sector', data=data)


In [ ]:
capital_results = np.concatenate(econ.capital_hist)
data = pd.DataFrame(capital_results, columns=['capital'])
data['sector'] = sector
data['steps'] = steps
sns.set_style('whitegrid')
sns.lineplot(x='steps', y='capital', hue='sector', data=data)

In [ ]:
data = np.concatenate([total_labor_results, total_capital_results, total_production_results])
data = pd.DataFrame(data, columns=['quantity'])
kind = ['labor'] * T + ['capital'] * T + ['production'] * T
data['kind'] = kind
steps = [t for t in range(T)] + [t for t in range(T)] + [t for t in range(T)]
data['steps'] = steps

sns.set_style('whitegrid')
sns.lineplot(x='steps', y='quantity', hue='kind', data=data)